### Building and Evaluating Regression Models

In this notebook we build and evaluate a linear regression model to predict ride rating from various ride, driver, and rider information. Thus `star_rating` is our label or Dependent Variable (DV). The general workflow will be the same for other regression algorithms; however, the particular details will differ. This notebook is based on material supplied by Cloudera under their Cloudera Academic Partner program and *Spark: The Definitive Guide* book by Bill Chambers and Matei Zaharia. 

Topics
- Preprocessing data (including `SQLTransformer`)
- Extracting, transforming, and selecting features
- Creating training and test datasets
- Specifying a linear regression model
- Use `explainParams` to explore the list of parameters for a machine learning class
- Fit linear regression model (using fit method)
- Examine performance of linear regression model (measures of fit, p-values,...)
- Examine model performance on test dataset

You can find details of all of the classes, methods, and attributes in the [Spark MLlib API Reference](https://spark.apache.org/docs/latest/api/python/reference/pyspark.ml.html) and a more general guide to their use in the [Machine Learning Library (MLlib) Guide](https://spark.apache.org/docs/latest/ml-guide.html)

In [0]:
# Read the enhanced ride data:
rides = spark.read.parquet("/mnt/cis442f-data/duocar/joined_all") 

In [0]:
rides.printSchema()

root
-- ride_id: string (nullable = true)
-- rider_id: string (nullable = true)
-- driver_id: string (nullable = true)
-- date_time: timestamp (nullable = true)
-- utc_offset: integer (nullable = true)
-- service: string (nullable = true)
-- origin_lat: decimal(9,6) (nullable = true)
-- origin_lon: decimal(9,6) (nullable = true)
-- dest_lat: decimal(9,6) (nullable = true)
-- dest_lon: decimal(9,6) (nullable = true)
-- distance: integer (nullable = true)
-- duration: integer (nullable = true)
-- cancelled: boolean (nullable = true)
-- star_rating: integer (nullable = true)
-- driver_birth_date: date (nullable = true)
-- driver_start_date: date (nullable = true)
-- driver_first_name: string (nullable = true)
-- driver_last_name: string (nullable = true)
-- driver_sex: string (nullable = true)
-- driver_ethnicity: string (nullable = true)
-- driver_student: boolean (nullable = true)
-- driver_home_block: string (nullable = true)
-- driver_home_lat: decimal(9,6) (nullable = true)
-- driver_home_lon: decimal(9,6) (nullable = true)
-- vehicle_make: string (nullable = true)
-- vehicle_model: string (nullable = true)
-- vehicle_year: integer (nullable = true)
-- vehicle_color: string (nullable = true)
-- vehicle_grand: boolean (nullable = true)
-- vehicle_noir: boolean (nullable = true)
-- vehicle_elite: boolean (nullable = true)
-- driver_rides: integer (nullable = true)
-- driver_stars: integer (nullable = true)
-- rider_birth_date: date (nullable = true)
-- rider_start_date: date (nullable = true)
-- rider_first_name: string (nullable = true)
-- rider_last_name: string (nullable = true)
-- rider_sex: string (nullable = true)
-- rider_ethnicity: string (nullable = true)
-- rider_student: boolean (nullable = true)
-- rider_home_block: string (nullable = true)
-- rider_home_lat: decimal(9,6) (nullable = true)
-- rider_home_lon: decimal(9,6) (nullable = true)
-- rider_work_lat: decimal(9,6) (nullable = true)
-- rider_work_lon: decimal(9,6) (nullable = true)
-- review: string (nullable = true)
-- driver_median_income: integer (nullable = true)
-- rider_median_income: integer (nullable = true)
-- Max_TemperatureF: integer (nullable = true)
-- Mean_TemperatureF: integer (nullable = true)
-- Min_TemperatureF: integer (nullable = true)
-- Max_Dew_PointF: integer (nullable = true)
-- MeanDew_PointF: integer (nullable = true)
-- Min_DewpointF: integer (nullable = true)
-- Max_Humidity: integer (nullable = true)
-- Mean_Humidity: integer (nullable = true)
-- Min_Humidity: integer (nullable = true)
-- Max_Sea_Level_PressureIn: decimal(4,2) (nullable = true)
-- Mean_Sea_Level_PressureIn: decimal(4,2) (nullable = true)
-- Min_Sea_Level_PressureIn: decimal(4,2) (nullable = true)
-- Max_VisibilityMiles: integer (nullable = true)
-- Mean_VisibilityMiles: integer (nullable = true)
-- Min_VisibilityMiles: integer (nullable = true)
-- Max_Wind_SpeedMPH: integer (nullable = true)
-- Mean_Wind_SpeedMPH: integer (nullable = true)
-- Max_Gust_SpeedMPH: integer (nullable = true)
-- PrecipitationIn: decimal(4,2) (nullable = true)
-- CloudCover: integer (nullable = true)
-- Events: string (nullable = true)
-- WindDirDegrees: integer (nullable = true)

#### Preprocess the data for modeling

Cancelled rides do not have a rating.  We could use the `filter` or `where` method to remove the cancelled rides. 
However, we will use the versatile `SQLTransformer` instead as we would be able to use it in a Spark MLlib pipeline.

In [0]:
# Check that all rides without a star_rating were cancelled rides
rides.groupBy("star_rating", "cancelled").count().orderBy("star_rating").show()


+-----------+---------+-----+
star_rating|cancelled|count|
+-----------+---------+-----+
 null| true| 2934|
 0| false| 256|
 1| false| 1286|
 2| false| 3155|
 3| false| 5564|
 4| false| 7195|
 5| false|28385|
+-----------+---------+-----+

**Note:**
- The `SQLTransformer` requires the `statement` keyword
- `__THIS__` represents the DataFrame passed to the `transform` method

In [0]:
# Use `SQLTransformer`to remove cancelled rides

from pyspark.ml.feature import SQLTransformer
processed = SQLTransformer(statement="SELECT * FROM __THIS__ WHERE cancelled == 0").transform(rides)
processed.count() 

Out[4]: 45841

In [0]:
# We could use the `filter` or `where` method to remove cancelled rides
# Check that we get the same number or rows
rides.filter(rides.cancelled == 0).count()

Out[5]: 45841

#### Extract, transform, and select features

Next we identify a few potential features to include in our model.  Let's create a function to explore potential (categorical) features.

In [0]:
 # Let us create a function to explore potential (categorical) features 
def explore(df, feature, label):
  from pyspark.sql.functions import count, mean, stddev
  aggregated = df \
    .groupBy(feature) \
    .agg(count(label), mean(label), stddev(label)).orderBy(feature)
  return aggregated

In [0]:
# Do all rides have reviews? No.... maybe people who submit reviews more often 
# for bad experiences than for good ones
rides.withColumn("reviewed", rides.review.isNotNull()).groupBy("reviewed").count().show()

+--------+-----+
reviewed|count|
+--------+-----+
 true| 1822|
 false|46953|
+--------+-----+

In [0]:
# Add new column as MLlib requires features to be in numeric form (not boolean)
from pyspark.sql.functions import col 

engineered1 = processed.withColumn("reviewed", col("review").isNotNull().cast("int"))

In [0]:
# Check that rides with a review have the categorical value set appropriately
engineered1.select("ride_id", "reviewed", "review").filter(engineered1.review.isNotNull()).show(5)

+----------+--------+--------------------+
 ride_id|reviewed| review|
+----------+--------+--------------------+
0000000009| 1|Dale is extremely...|
0000000037| 1| Very junky car.|
0000000071| 1|most awful stench...|
0000000083| 1| No trouble of note.|
0000000086| 1|The driver drove ...|
+----------+--------+--------------------+
only showing top 5 rows

In [0]:
# Check that rides without a review have the categorical value set appropriately
engineered1.select("ride_id", "reviewed", "review").filter(engineered1.review.isNull()).show(5)

+----------+--------+------+
 ride_id|reviewed|review|
+----------+--------+------+
0000000001| 0| null|
0000000002| 0| null|
0000000003| 0| null|
0000000004| 0| null|
0000000005| 0| null|
+----------+--------+------+
only showing top 5 rows

In [0]:
# Our "domain knowledge" tells us that people may be more likely to add
# a review after having a bad experience
# Did the rider review the ride?
aggregated = explore(engineered1, "reviewed", "star_rating")

aggregated.show()
aggregated.createOrReplaceTempView("agg_view")

+--------+------------------+-----------------+------------------------+
reviewed|count(star_rating)| avg(star_rating)|stddev_samp(star_rating)|
+--------+------------------+-----------------+------------------------+
 0| 44019|4.307889774869943| 1.081740043694224|
 1| 1822|2.944017563117453| 1.5765806972412695|
+--------+------------------+-----------------+------------------------+

In [0]:
%sql
SELECT * FROM agg_view

-- Average rating is higher for rides without reviews
-- So, even the presence or absence of a review might be predictive in our model

reviewed,count(star_rating),avg(star_rating),stddev_samp(star_rating)
0,44019,4.307889774869943,1.081740043694224
1,1822,2.944017563117453,1.5765806972412695


In [0]:
# Does the year of the vehicle matter?
aggregated = explore(engineered1, "vehicle_year", "star_rating")
aggregated.show()

+------------+------------------+------------------+------------------------+
vehicle_year|count(star_rating)| avg(star_rating)|stddev_samp(star_rating)|
+------------+------------------+------------------+------------------------+
 2002| 1662| 3.786401925391095| 1.3238816650905982|
 2003| 1405| 3.706049822064057| 1.4050171031678058|
 2004| 1543|3.9228775113415426| 1.278329411969232|
 2005| 1656| 4.032004830917875| 1.2165286019044412|
 2006| 1486|4.0598923283983845| 1.2176230039075355|
 2007| 2883| 4.002428026361429| 1.2458094926970944|
 2008| 1302| 4.100614439324117| 1.1930555879797213|
 2009| 1782| 4.074635241301908| 1.2066337118622132|
 2010| 2040| 4.154901960784314| 1.184966104640845|
 2011| 2373|4.1597134428992835| 1.1962681934668016|
 2012| 2019| 4.214462605250124| 1.128980721157208|
 2013| 5293|4.3784243340260725| 1.042648386272745|
 2014| 4721| 4.398856174539293| 1.035385973980204|
 2015| 6208| 4.428157216494846| 1.0247428814212247|
 2016| 5570| 4.460861759425494| 0.985618243844526|
 2017| 3898|4.4781939456131346| 0.979209917198152|
+------------+------------------+------------------+------------------------+

The age of the vehicle looks promising as predictor

In [0]:
display(aggregated)

vehicle_year,count(star_rating),avg(star_rating),stddev_samp(star_rating)
2002,1662,3.786401925391095,1.3238816650905982
2003,1405,3.706049822064057,1.4050171031678058
2004,1543,3.9228775113415426,1.278329411969232
2005,1656,4.032004830917875,1.2165286019044412
2006,1486,4.0598923283983845,1.2176230039075355
2007,2883,4.002428026361429,1.2458094926970944
2008,1302,4.100614439324117,1.1930555879797213
2009,1782,4.074635241301908,1.2066337118622132
2010,2040,4.154901960784314,1.184966104640845
2011,2373,4.1597134428992835,1.1962681934668016


In [0]:
# What about the color of the vehicle?
aggregated = explore(engineered1, "vehicle_color", "star_rating").sort("avg(star_rating)", ascending=False)
aggregated.show()

+-------------+------------------+------------------+------------------------+
vehicle_color|count(star_rating)| avg(star_rating)|stddev_samp(star_rating)|
+-------------+------------------+------------------+------------------------+
 black| 20892| 4.387421022400919| 1.0498150497725625|
 other| 972| 4.300411522633745| 1.0893548366982366|
 brown| 1018| 4.287819253438114| 1.1089467922075988|
 gray| 4678| 4.19880290722531| 1.1393324942493839|
 silver| 4041| 4.186834941846078| 1.1773375336096665|
 white| 6486| 4.143231575701511| 1.1964956455364586|
 blue| 2797| 4.139792634966035| 1.2035862422506949|
 green| 1279| 4.132134480062549| 1.1630289608378617|
 red| 2365|4.1158562367864695| 1.2091472067035778|
 yellow| 1313|3.6207159177456205| 1.3960736883018343|
+-------------+------------------+------------------+------------------------+

In [0]:
display(aggregated)
# Looks like black cars are preferred (but may already be correlated with Black Car and Elite services)
# Is there a distinct dislike of yellow cars? 

vehicle_color,count(star_rating),avg(star_rating),stddev_samp(star_rating)
black,20892,4.387421022400919,1.0498150497725625
other,972,4.300411522633745,1.0893548366982366
brown,1018,4.287819253438114,1.1089467922075988
gray,4678,4.19880290722531,1.1393324942493839
silver,4041,4.186834941846078,1.1773375336096665
white,6486,4.143231575701511,1.1964956455364586
blue,2797,4.139792634966035,1.2035862422506949
green,1279,4.132134480062549,1.1630289608378617
red,2365,4.1158562367864695,1.2091472067035778
yellow,1313,3.6207159177456205,1.3960736883018343


In [0]:
# Do riders give better reviews on sunny days?
aggregated = explore(engineered1, "CloudCover", "star_rating").sort("CloudCover")
aggregated.show()

+----------+------------------+-----------------+------------------------+
CloudCover|count(star_rating)| avg(star_rating)|stddev_samp(star_rating)|
+----------+------------------+-----------------+------------------------+
 0| 10349|4.301961542177988| 1.1107658402665925|
 1| 6399| 4.29957805907173| 1.1029181263206234|
 2| 4776|4.319723618090452| 1.092840291467438|
 3| 9073|4.238950732943899| 1.1425104445249084|
 4| 2214| 4.28590785907859| 1.122022157216512|
 5| 1094|4.194698354661791| 1.1523988467425252|
 6| 6716|4.191036331149494| 1.173414894586613|
 7| 4269|4.142890606699461| 1.2000894931008714|
 8| 951|4.160883280757098| 1.218448045690291|
+----------+------------------+-----------------+------------------------+

In [0]:
display(aggregated)
# It does not look like cloud cover makes much differenct to ratings

CloudCover,count(star_rating),avg(star_rating),stddev_samp(star_rating)
0,10349,4.301961542177988,1.1107658402665925
1,6399,4.29957805907173,1.1029181263206234
2,4776,4.319723618090452,1.092840291467438
3,9073,4.238950732943899,1.1425104445249084
4,2214,4.28590785907859,1.122022157216512
5,1094,4.194698354661791,1.1523988467425252
6,6716,4.191036331149494,1.173414894586613
7,4269,4.142890606699461,1.2000894931008714
8,951,4.160883280757098,1.218448045690291


Spark MLlib algorithms require the features to be a vector of doubles.  As a result, we need to further transform these features before we can build our regression model.

1. Use `StringIndexer` to convert `vehicle_color` into a numeric index (note that it is an estimator with a fit method that produces a transformer)
2. Use `OneHotEncoder` to generate a set of corresponding dummy variables (since the ordering of the vehicle color index is meaningless)
3. Use `VectorAssembler` to assemble the features into a single column of vectors

In [0]:
# Use `StringIndexer` (an estimator) to convert `vehicle_color` 
# from string codes to numeric codes
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="vehicle_color", outputCol="vehicle_color_ix")
indexer_model = indexer.fit(engineered1)
print (list(enumerate(indexer_model.labels)))
indexed = indexer_model.transform(engineered1)
indexed.select("vehicle_color", "vehicle_color_ix").show(5)

# Note that the numbers imply an ordering. However, such an ordering is 
# meaningless and therefore this should not be used in the model


[(0, 'black'), (1, 'white'), (2, 'gray'), (3, 'silver'), (4, 'blue'), (5, 'red'), (6, 'yellow'), (7, 'green'), (8, 'brown'), (9, 'other')]
+-------------+----------------+
vehicle_color|vehicle_color_ix|
+-------------+----------------+
 white| 1.0|
 black| 0.0|
 white| 1.0|
 yellow| 6.0|
 yellow| 6.0|
+-------------+----------------+
only showing top 5 rows

In [0]:
# Use `OneHotEncoderEstimator` to generate a set of dummy variables
# Note that this was `OneHotEncoderEstimator` in Spark 2.x
from pyspark.ml.feature import OneHotEncoder
ohe = OneHotEncoder(inputCols=["vehicle_color_ix"], outputCols=["vehicle_color_cd"])
encoded = ohe.fit(indexed).transform(indexed)
encoded.select("vehicle_color", "vehicle_color_ix", "vehicle_color_cd").show(5)

+-------------+----------------+----------------+
vehicle_color|vehicle_color_ix|vehicle_color_cd|
+-------------+----------------+----------------+
 white| 1.0| (9,[1],[1.0])|
 black| 0.0| (9,[0],[1.0])|
 white| 1.0| (9,[1],[1.0])|
 yellow| 6.0| (9,[6],[1.0])|
 yellow| 6.0| (9,[6],[1.0])|
+-------------+----------------+----------------+
only showing top 5 rows

In [0]:
# Now we are ready to select the features (and label)
selected = encoded.select("reviewed", "vehicle_year", "vehicle_color_cd", "star_rating")
features = ["reviewed", "vehicle_year", "vehicle_color_cd"]

In [0]:
# Finally, we must assemble the features into a single column of vectors
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=features, outputCol="features")
assembled = assembler.transform(selected)

for item in assembled.where(assembled.reviewed == 1).orderBy(assembled.star_rating).head(2):
    print (item)
print("_________________________________")
for item in assembled.where(assembled.reviewed == 0).head(2):
    print (item)

Row(reviewed=1, vehicle_year=2007, vehicle_color_cd=SparseVector(9, {1: 1.0}), star_rating=1, features=SparseVector(11, {0: 1.0, 1: 2007.0, 3: 1.0}))
Row(reviewed=1, vehicle_year=2013, vehicle_color_cd=SparseVector(9, {4: 1.0}), star_rating=1, features=SparseVector(11, {0: 1.0, 1: 2013.0, 6: 1.0}))
_________________________________
Row(reviewed=0, vehicle_year=2015, vehicle_color_cd=SparseVector(9, {1: 1.0}), star_rating=5, features=SparseVector(11, {1: 2015.0, 3: 1.0}))
Row(reviewed=0, vehicle_year=2015, vehicle_color_cd=SparseVector(9, {0: 1.0}), star_rating=5, features=SparseVector(11, {1: 2015.0, 2: 1.0}))

The three fields being assembled are 

`reviewed=1, vehicle_year=2007, vehicle_color_cd=SparseVector(9, {1: 1.0})`

 - `vehicle_color_cd` is already a SparseVector with 9 elements
 - This example has a value of 1.0 for the 1st element. The rest of the elements have value 0.0 (SparseVectors omits such elements)
 - Each element represents one of the 10 vehicle colors found in the dataset
 - The 10th color is represented by all elements having values of 0.0

The assembled version in features looks like

`features=SparseVector(11, {0: 1.0, 1: 2007.0, 3: 1.0}))`

 - There are now 11 elements in the vector. Nine for the vehicle colors and one each for `reviewed` and `vehicle_year`
 - The 0th element with value 1.0 corresponds to `reviewed=1` (Note that examples with `reviewed=0` do not contain a 0th element)
 - The 1st element with value 2007.0 corresponds to `vehicle_year=2007`
 - The 3rd element with value 1.0 corresponds to `vehicle_color_cd=SparseVector(9, {1: 1.0}`

#### Create train and test sets

In [0]:
# Use `randomSplit` method to create train and test sets
(train, test) = assembled.randomSplit([0.7, 0.3], 12345)

#### Specify a linear regression model

The `explainParams` shows us the paramters available to the data scientist. 
See [this article](https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/) for accessible description of the ideas behind regularization and elesticNet parameters.

In [0]:
# Use the `LinearRegression` class to specify a linear regression model
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="features", labelCol="star_rating")

# Use the `explainParams` method to get a full list of parameters
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: star_rating)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0. (default: 0.0)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (default: auto)
standardization: whether to standardize the training features before fitting the model. (default: True)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

#### Fit the linear regression model

In [0]:
# Use the `fit` method to fit the linear regression model:
lr_model = lr.fit(train)

In [0]:
# Query model performance:
print ("R-Squared: " + str(lr_model.summary.r2))
print ("________________________________________________________________")
# Query the model:
print ("Intercept: " + str(lr_model.intercept))
print ("________________________________________________________________")
print ("Coefficients: " + str(lr_model.coefficients))
print ("________________________________________________________________")
print ("Coef.SE: "+ str(lr_model.summary.coefficientStandardErrors))
print ("________________________________________________________________")
print ("t-values: "+ str(lr_model.summary.tValues))
print ("________________________________________________________________")
print ("p-values: "+ str(lr_model.summary.pValues))

R-Squared: 0.09233264758083592
________________________________________________________________
Intercept: -80.46167444415252
________________________________________________________________
Coefficients: [-1.266691157477927,0.042137004959463625,0.06335202789961905,-0.02846245032657166,-0.0006602845385864919,-0.013477591871998693,-0.023466579076631616,0.03848631061920815,-0.4863009346560902,-0.004097099207801906,0.008879373776285886]
________________________________________________________________
Coef.SE: [0.03061878382748393, 0.0015102087302818325, 0.04318879659607232, 0.04564666171196382, 0.04660478335192778, 0.047262113134280866, 0.04934618097462677, 0.050811418801105415, 0.05524755564675362, 0.055802658590096706, 0.05869695459599073, 3.0420113724664466]
________________________________________________________________
t-values: [-41.3697410261254, 27.9014444258974, 1.466862540582583, -0.6235384858190354, -0.014167741830285316, -0.2851669334738849, -0.4755500550021866, 0.7574342840111931, -8.802216296508053, -0.07342121883291448, 0.15127486319183564, -26.450155700409045]
________________________________________________________________
p-values: [0.0, 0.0, 0.1424232722368255, 0.5329351607808914, 0.9886962439593989, 0.7755180964410111, 0.6343982083488424, 0.4487952958760797, 0.0, 0.9414713915293236, 0.8797598502026669, 0.0]

#### Evaluate model performance on the test set

**Method 1: Use the `evaluate` method of the `LinearRegressionModel` class**

In [0]:
summary_test = lr_model.evaluate(test)
print("R2: " + str(summary_test.r2))
print ("RMSE: " + str(summary_test.rootMeanSquaredError))
type (summary_test)

R2: 0.09445675782345075
RMSE: 1.0829997589339142
Out[26]: pyspark.ml.regression.LinearRegressionSummary

In [0]:
type(summary_test.predictions)

Out[27]: pyspark.sql.dataframe.DataFrame

In [0]:
# The predictions are included in the summary
# Formatting and name changes made to make printed 
# version of the table readable
from pyspark.sql.functions import round
summary_test.predictions \
    .withColumnRenamed("vehicle_year", "year") \
    .withColumnRenamed("vehicle_color_cd", "color") \
    .withColumnRenamed("star_rating", "stars") \
    .withColumnRenamed("prediction", "pred") \
    .withColumn("prediction", round(col("pred"), 3)) \
    .drop("pred") \
    .show(5, False)
print("Number of rows: " + str(summary_test.predictions.count()) )

+--------+----+-------------+-----+-----------------------+----------+
reviewed|year|color |stars|features |prediction|
+--------+----+-------------+-----+-----------------------+----------+
0 |2002|(9,[0],[1.0])|2 |(11,[1,2],[2002.0,1.0])|3.96 |
0 |2002|(9,[0],[1.0])|2 |(11,[1,2],[2002.0,1.0])|3.96 |
0 |2002|(9,[0],[1.0])|3 |(11,[1,2],[2002.0,1.0])|3.96 |
0 |2002|(9,[0],[1.0])|3 |(11,[1,2],[2002.0,1.0])|3.96 |
0 |2002|(9,[0],[1.0])|3 |(11,[1,2],[2002.0,1.0])|3.96 |
+--------+----+-------------+-----+-----------------------+----------+
only showing top 5 rows

Number of rows: 13805

In [0]:
# Create a sample for plotting  
prediction_sample = summary_test.predictions.select("prediction", "star_rating").sample(True, 0.1)
# prediction_sample.createOrReplaceTempView("predictions")
prediction_sample.show(5)

print("Number of rows: " + str(prediction_sample.count()) )

+------------------+-----------+
 prediction|star_rating|
+------------------+-----------+
3.9599615125932814| 5|
3.9599615125932814| 5|
3.9599615125932814| 5|
3.9599615125932814| 5|
 3.868147034367084| 3|
+------------------+-----------+
only showing top 5 rows

Number of rows: 1385

In [0]:
display(prediction_sample)

prediction,star_rating
3.9599615125932814,5
3.9599615125932814,5
3.9599615125932814,5
3.9599615125932814,5
3.868147034367084,3
3.868147034367084,3
3.868147034367084,3
3.868147034367084,3
3.868147034367084,3
3.868147034367084,4


**Method 2: Use the `evaluate` method of the `RegressionEvaluator` class**

In [0]:
# Generate predictions on the test set:
# Feel free to change format of table if you need to print it out
test_with_predictions = lr_model.transform(test)
test_with_predictions.show(5, False)

+--------+------------+----------------+-----------+-----------------------+------------------+
reviewed|vehicle_year|vehicle_color_cd|star_rating|features |prediction |
+--------+------------+----------------+-----------+-----------------------+------------------+
0 |2002 |(9,[0],[1.0]) |2 |(11,[1,2],[2002.0,1.0])|3.9599615125932814|
0 |2002 |(9,[0],[1.0]) |2 |(11,[1,2],[2002.0,1.0])|3.9599615125932814|
0 |2002 |(9,[0],[1.0]) |3 |(11,[1,2],[2002.0,1.0])|3.9599615125932814|
0 |2002 |(9,[0],[1.0]) |3 |(11,[1,2],[2002.0,1.0])|3.9599615125932814|
0 |2002 |(9,[0],[1.0]) |3 |(11,[1,2],[2002.0,1.0])|3.9599615125932814|
+--------+------------+----------------+-----------+-----------------------+------------------+
only showing top 5 rows

In [0]:
# Create instance of `RegressionEvaluator` class:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="star_rating", metricName="r2")
print(evaluator.explainParams())

labelCol: label column name. (default: label, current: star_rating)
metricName: metric name in evaluation - one of:
 rmse - root mean squared error (default)
 mse - mean squared error
 r2 - r^2 metric
 mae - mean absolute error
 var - explained variance. (default: rmse, current: r2)
predictionCol: prediction column name. (default: prediction, current: prediction)
throughOrigin: whether the regression is through the origin. (default: False)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

In [0]:
# Evaluate using the `RegressionEvaluator` instance
evaluator.evaluate(test_with_predictions)

Out[34]: 0.09445675782345075

In [0]:
# Evaluate using another metric:
evaluator.setMetricName("rmse").evaluate(test_with_predictions)

Out[35]: 1.0829997589339142

###Hands On

![Hands-on](https://cis442f-open-data.s3.amazonaws.com/pictures/hands.png "Hands-on")



#### Exercises

(1)  Experiment with different sets of features (could use Random Forest to help with Feature Selection)

(2)  Experiment with a different regression method.


#### References 
These links are for latest version of Spark. You might have to look at the documentation for the specific version you are using as MLlib is changing quite quickly.

[Linear Regression Estimator](http://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegression.html)

[Linear Regression Model](http://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegressionModel.html) created by the Linear Regression Estimator

[Linear Regression Training Summary](http://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegressionTrainingSummary.html) training results

[Linear Regression Summary](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegressionSummary.html) from evaluating a model on a dataset

[Classification and regression](https://spark.apache.org/docs/latest/ml-classification-regression.html)

In [0]:
# Playing around with rounding of output for display in notebook
from pyspark.sql.functions import round
# test_with_predictions.select([round(avg(c), 3).alias(c) for c in test_with_predictions.columns]).show(5,False)

test_with_predictions.select([round(c, 5).alias(c) for c in ["prediction"]]).show(5,False)

+----------+
prediction|
+----------+
3.95996 |
3.95996 |
3.95996 |
3.95996 |
3.95996 |
+----------+
only showing top 5 rows